In [17]:
#rdkit imports
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

#ml imports
#from sklearn.preprocessing import StandardScaler
#from sklearn import cross_validation
#from sklearn import cross_validation
#from sklearn.kernel_ridge import KernelRidge
#from sklearn.linear_model import Ridge, LinearRegression
#from sklearn.svm import SVR
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.neural_network import MLPRegressor
#from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
#from sklearn.model_selection import GridSearchCV

#housekeeping imports
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp

Maximum common substructure

In [193]:
# finds maximum common substructure among list of molecule objects
def get_mcs(input_df):      # gets a df containing SMILES strings of products
    
    mol_list = list(input_df['SMILES']) # converts product column to list

    #for i in range(len(mol_list)): # converts SMILES strings to molecule objects
        #if mol_list[i] = 'none'
        #mol_list[i] = Chem.MolFromSmiles(mol_list[i])

    #maximum_common_substructure = rdFMCS.FindMCS(mol_list)
    
    return mol_list #maximum_common_substructure

#Draw.MolToImage(maximum_common_substructure, size=(300, 300), kekulize=False, wedgeBonds=True)

Data cleaning

In [194]:
input_df = pd.read_csv('~/OneDrive/Documents/Python/CLASSES/DIRECT/metamoles/datasets/playground_df_cleaned_kegg_with_smiles.csv')

In [195]:
def input_data(input_df): #cleans input df and returns neccessary elements
    for index, row in input_df.iterrows():
        if row['SMILES'] == 'none':
            input_df.drop(index, inplace=True)            
        
    return input_df

In [196]:
input_data(input_df).head()

,Unnamed: 0,entry,KEGG,PubChem,CID,SMILES
4,5,1.1.1.110,C03964,6685,440177,C1=CC(=CC=C1C[C@H](C(=O)O)O)O
5,7,1.1.1.110,C00331,3625,803,C1=CC=C2C(=C1)C(=CN2)CC(=O)C(=O)O
6,8,1.1.1.110,C01179,4406,979,C1=CC(=CC=C1CC(=O)C(=O)O)O
7,11,1.1.1.110,C05607,7930,643327,C1=CC=C(C=C1)C[C@H](C(=O)O)O
8,16,1.1.1.153,C02953,5871,135398687,C[C@@H]([C@@H](C1=NC2=C(NC1)N=C(NC2=O)N)O)O


In [197]:
def fingerprint_products(input_df): #fingerprints all products in a given df
    
    mol_list = []
    fp_list = []
    
    for index, row in input_df.iterrows():
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES'])) #get mols from SMILES and add mols to list
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))) #get fingerprints from mols and and fingerprints to list
        
    input_df.insert(6, column='Mol', value=mol_list)
    input_df.insert(7, column='Fingerprint', value= fp_list)
            
    return input_df

In [198]:
input_df2 = fingerprint_products(input_df) #build df containing new Mol and Fingerprint columns

split by enzyme

In [205]:
def split_by_enzyme(input_df):
    unique_enzymes = set(input_df['entry'].unique())
    
    enzyme_df_list = []
    
    for entry in unique_enzymes:
        
        #make new enzyme_df
        
        #for row in input_df #fill enzmye df
            #if row['entry'] is entry 
                #add row to enzyme_df
        
        enzyme_df_list.append(enzyme_df)
    
    return enzyme_df_list

In [204]:
def main(input_df):
    
    input_df = fingerprint_products(input_df)    #expand input df: generate mols from SMILES then generate fingerprints from mols, adding columns for each
    
    enzyme_df_list = split_by_enzyme(input_df)    #split expanded df by rows, grouped by enzyme entry (1.1.1.110 etc), into a list of dataframes
    
    master_df = pd.DataFrame()    #initialize output
    
    for enzyme_df in enzyme_df_list:    #loop through list of enzyme dataframes
        
        metric = sim_metric(input_df)    #get similarity matrix dataframe
        
        #for index, row in metric.iterrows():    #loop through similarity matrix
            #enzyme_df['Dist'].loc[index] = metric.[0]loc[index]
            
        #add averaged distance calculated from metric
    
    # concatenate enzyme df onto master_df     
    
    return master_df
        
        

similarity functions

In [200]:
def sim_i_j(row_i, row_j):
    """DocString"""
    return DataStructs.FingerprintSimilarity(row_i['Fingerprint'], row_j['Fingerprint'], metric=DataStructs.TanimotoSimilarity)

In [201]:
def sim_i_all(input_df, index_i, row_i, metric):
    """DocString"""
    for index_j, row_j in input_df.iterrows():
        if index_j < index_i: #skip redundant rows
            continue
        elif index_i == index_j: #autocorrelate rows
            metric.loc[index_i, index_j] = 1
        else:
            metric.loc[index_i, index_j] = sim_i_j(row_i, row_j) #fill matrix with calculated similarity at two positions at once 
            metric.loc[index_j, index_i] = metric.loc[index_i, index_j]
    return 

In [202]:
def sim_metric(input_df):
    """DocString"""
    metric = pd.DataFrame()
    for index_i, row_i in input_df.iterrows():
        sim_i_all(input_df, index_i, row_i, metric)
    return metric

In [203]:
sim_metric(input_df2).head()

,4,5,6,7,8,9,10,11,12,13,...,40,41,42,43,44,45,46,47,48,49
4,1.000000,0.393316,0.662447,0.842640,0.378661,0.382845,0.376569,0.358921,0.385069,0.385069,...,0.320856,0.461538,0.310160,0.376569,0.382845,0.358921,0.378661,0.393162,0.391026,0.391026
5,0.393316,1.000000,0.472826,0.373656,0.366265,0.372315,0.349939,0.356618,0.401452,0.401452,...,0.334711,0.484375,0.331950,0.349939,0.372315,0.356618,0.366265,0.389163,0.388409,0.388409
6,0.662447,0.472826,1.000000,0.538136,0.361570,0.371369,0.359504,0.353306,0.385069,0.385069,...,0.302083,0.436975,0.291667,0.359504,0.371369,0.353306,0.361570,0.396146,0.394004,0.394004
7,0.842640,0.373656,0.538136,1.000000,0.322105,0.323529,0.320000,0.302714,0.324165,0.324165,...,0.295455,0.429825,0.298851,0.320000,0.323529,0.302714,0.322105,0.326923,0.324786,0.324786
8,0.378661,0.366265,0.361570,0.322105,1.000000,0.814624,0.512753,0.558690,0.363256,0.363256,...,0.341176,0.492188,0.328125,0.512753,0.814624,0.558690,1.000000,0.265614,0.266571,0.266571
